<a href="https://colab.research.google.com/github/hcorbel/fine_tuning_bert_stackoverflow/blob/main/Stackoverflow_fine_tune_bert__for_multi_label_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we are going to fine-tune a bert-base-uncased model to predict one or more labels for a given piece of text. This notebook is from https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
The model adds a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.

Dataset is from stackoverflow questions: body, title and tags that have been preprocessed. Only questions labelled by the 100 more frequent tags are kept (df_final.csv).

In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd


In [ ]:
df3=pd.read_csv("/gdrive/MyDrive/df_final.csv", sep=',')

In [ ]:
df3.head()

,Unnamed: 0,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount,concat,clean_tagB,clean_tag,clean_tag_list,clean_concat,clean_concat_list,nb_tags,Tags_list2,number_of_tags
0,0,How to deploy a Java EE application in the Ama...,<p>Here's the thing: I need to take a certain ...,<java><jakarta-ee><amazon-ec2><cloud><cloud-ho...,4042477,22,19503,0.0,3,How to deploy a Java EE application in the Ama...,"['java', 'jakarta-ee', 'amazon-ec2', 'cloud', ...",java jakarta-ee amazon-ec2 cloud cloud-hosting,"['java', 'jakarta-ee', 'amazon-ec2', 'cloud', ...",deploy java ee application amazon cloud java e...,"['deploy', 'java', 'ee', 'application', 'amazo...",5,['java'],1
1,1,How to save Bitmap as icon?,<p>I need to save Bitmap object loaded from im...,<c#><.net><vb.net><icons><bitmap>,4042488,7,10894,0.0,1,How to save Bitmap as icon? <p>I need to save ...,"['c#', '.net', 'vb.net', 'icon', 'bitmap']",c# .net vb.net icon bitmap,"['c#', '.net', 'vb.net', 'icon', 'bitmap']",save bitmap icon save bitmap object loaded ima...,"['save', 'bitmap', 'icon', 'save', 'bitmap', '...",5,"['c#', '.net']",2
2,2,Why can't I change the value of String.Empty?,<p>While I understand that changing the value ...,<c#><.net><reflection><constants><immutability>,19964254,22,992,0.0,1,Why can't I change the value of String.Empty? ...,"['c#', '.net', 'reflection', 'constant', 'immu...",c# .net reflection constant immutability,"['c#', '.net', 'reflection', 'constant', 'immu...",value string.empty understand value idea under...,"['value', 'string.empty', 'understand', 'value...",5,"['c#', '.net']",2
3,3,What is the equivalent of Perl's (<>) in Pytho...,<p>In Perl one uses:</p>\n\n<pre><code>while (...,<python><perl><file-io><file><command-line-arg...,4554819,7,881,0.0,7,What is the equivalent of Perl's (<>) in Pytho...,"['python', 'perl', 'file-io', 'file', 'command...",python perl file-io file command-line-arguments,"['python', 'perl', 'file-io', 'file', 'command...",equivalent perls python fileinput expected per...,"['equivalent', 'perls', 'python', 'fileinput',...",5,"['python', 'file']",2
4,4,Getting the subversion repository number into ...,<p>I'd like to implement a way of recording th...,<svn><debugging><testing><revision><versions>,16248,40,48681,0.0,6,Getting the subversion repository number into ...,"['svn', 'debugging', 'testing', 'revision', 'v...",svn debugging testing revision version,"['svn', 'debugging', 'testing', 'revision', 'v...",subversion repository number code id implement...,"['subversion', 'repository', 'number', 'code',...",5,"['debugging', 'testing']",2


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46732 entries, 0 to 46731
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         46732 non-null  int64  
 1   Title              46732 non-null  object 
 2   Body               46732 non-null  object 
 3   Tags               46732 non-null  object 
 4   Id                 46732 non-null  int64  
 5   Score              46732 non-null  int64  
 6   ViewCount          46732 non-null  int64  
 7   FavoriteCount      40590 non-null  float64
 8   AnswerCount        46732 non-null  int64  
 9   concat             46732 non-null  object 
 10  clean_tagB         46732 non-null  object 
 11  clean_tag          46732 non-null  object 
 12  clean_tag_list     46732 non-null  object 
 13  clean_concat       46732 non-null  object 
 14  clean_concat_list  46732 non-null  object 
 15  nb_tags            46732 non-null  int64  
 16  Tags_list2         467

In [ ]:
df3.sample(5)


,Unnamed: 0,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount,concat,clean_tagB,clean_tag,clean_tag_list,clean_concat,clean_concat_list,nb_tags,Tags_list2,number_of_tags
40904,43783,Injection of autowired dependencies failed; ne...,<p>I am new here and I am trying to create an ...,<java><spring><hibernate><maven><spring-mvc>,42832387,6,12110,NaN,4,Injection of autowired dependencies failed; ne...,"['java', 'spring', 'hibernate', 'maven', 'spri...",java spring hibernate maven spring-mvc,"['java', 'spring', 'hibernate', 'maven', 'spri...",injection autowired dependency failed nested e...,"['injection', 'autowired', 'dependency', 'fail...",5,"['java', 'spring', 'hibernate', 'maven', 'spri...",5
39406,42189,Defining variables in SVG,<p>I currently have a line filled in SVG like ...,<xml><variables><path><svg><definition>,6556656,6,3066,0.0,1,Defining variables in SVG <p>I currently have ...,"['xml', 'variable', 'path', 'svg', 'definition']",xml variable path svg definition,"['xml', 'variable', 'path', 'svg', 'definition']",defining variable svg currently line filled sv...,"['defining', 'variable', 'svg', 'currently', '...",5,['xml'],1
20226,21686,How to make generic computations over heteroge...,<p><strong>PREMISE:</strong></p>\n\n<p>After p...,<c++><c++11><iteration><variadic-templates><te...,14261183,77,7300,0.0,5,How to make generic computations over heteroge...,"['c++', 'c++11', 'iteration', 'variadic-templa...",c++ c++11 iteration variadic-templates templat...,"['c++', 'c++11', 'iteration', 'variadic-templa...",generic computation heterogeneous argument pac...,"['generic', 'computation', 'heterogeneous', 'a...",5,"['c++', 'c++11']",2
23957,25660,Glassfish 3 security - Form based authenticati...,<p>I want to understand form based security a...,<java><security><jsf><jakarta-ee><glassfish>,7944963,24,26274,0.0,4,Glassfish 3 security - Form based authenticati...,"['java', 'security', 'jsf', 'jakarta-ee', 'gla...",java security jsf jakarta-ee glassfish,"['java', 'security', 'jsf', 'jakarta-ee', 'gla...",glassfish security form based authentication j...,"['glassfish', 'security', 'form', 'based', 'au...",5,"['java', 'security']",2
35497,38016,Yocto Bitbake - Git Fetch always stuck,<p>I am using YOCTO to build an image but the ...,<linux><linux-kernel><yocto><bitbake><openembe...,36850974,6,2974,NaN,3,Yocto Bitbake - Git Fetch always stuck <p>I am...,"['linux', 'linux-kernel', 'yocto', 'bitbake', ...",linux linux-kernel yocto bitbake openembedded,"['linux', 'linux-kernel', 'yocto', 'bitbake', ...",yocto bitbake git fetch always stuck yocto bui...,"['yocto', 'bitbake', 'git', 'fetch', 'always',...",5,['linux'],1


DATASET PREPARATION

On va créer un jeu d'entrainement, de test et de validation. On a 1 colonne par tag (n=100 colonnes + 1 colonne de texte = [corps + titre]) à partir de 10000 questions sélectionnées au hasard dans le dataaset

In [ ]:
df3=df3.sample(10000)
df3=df3[['Tags_list2', 'clean_concat']]

In [ ]:
df3.head()

,Tags_list2,clean_concat
6475,"['c++', 'algorithm']",iterate equal value standard library suppose v...
1652,"['c++', 'optimization']",integer division float multiplication calculat...
344,"['c#', '.net']",create msbuild custom task modify c# code comp...
34693,['javascript'],share build script lerna package lerna repo co...
16797,"['iphone', 'io']",url add home screen iphone safari customized a...


In [ ]:
df3['Tags_list2']=df3['Tags_list2'].str.replace("[","")
df3['Tags_list2']=df3['Tags_list2'].str.replace("]","")
df3['Tags_list2']=df3['Tags_list2'].str.replace("'","")
df3['Tags_list2']=df3['Tags_list2'].str.replace(",","")
corpustag=" ".join(df3.Tags_list2.values)
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_tag = CountVectorizer (tokenizer = lambda x: x.split())
tag_tf = vectorizer_tag.fit_transform([corpustag])
print("Number of tags :", tag_tf.shape[1])
count_array = tag_tf.toarray()
b = pd.DataFrame(data=count_array,columns = vectorizer_tag.get_feature_names_out())
print(b)


<ipython-input-10-f79f7998f8ba>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df3['Tags_list2']=df3['Tags_list2'].str.replace("[","")
<ipython-input-10-f79f7998f8ba>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df3['Tags_list2']=df3['Tags_list2'].str.replace("]","")


Number of tags : 100
   .net  ajax  algorithm  amazon-web-services  android  android-studio  \
0   703   118        138                  103      694              67   

   angular  angularjs  array  asp.net  ...  ubuntu  unit-testing  \
0      118         96    164      273  ...      64           164   

   visual-studio  web-services  window  winforms  wpf  xaml  xcode  xml  
0            161            64     194        91  154    68    217  113  

[1 rows x 100 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
df4=df3
df4['Tags_list2']=df4['Tags_list2'].str.replace("[","")
df4['Tags_list2']=df4['Tags_list2'].str.replace("]","")
df4['Tags_list2']=df4['Tags_list2'].str.replace("'","")
df4['Tags_list2']=df4['Tags_list2'].str.replace(",","")
mlb = MultiLabelBinarizer()
df5 = (pd.DataFrame(mlb.fit_transform(df4['Tags_list2'].str.split()),
                    columns=mlb.classes_, index=df4['clean_concat']).eq(1))
df5.head()

<ipython-input-11-aced475fdd97>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df4['Tags_list2']=df4['Tags_list2'].str.replace("[","")
<ipython-input-11-aced475fdd97>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df4['Tags_list2']=df4['Tags_list2'].str.replace("]","")


,.net,ajax,algorithm,amazon-web-services,android,android-studio,angular,angularjs,array,asp.net,...,ubuntu,unit-testing,visual-studio,web-services,window,winforms,wpf,xaml,xcode,xml
clean_concat,,,,,,,,,,,,,,,,,,,,,
iterate equal value standard library suppose vector vector sorted equal element next iterator pair representing range equal element standard library id value code example contains number id point element loop elegant solve standard library,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
integer division float multiplication calculate fraction int value say faster conversion factor decide many division done time multiplication edit comment clear floating point math slower much replace multiplication n division n worth anymore,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
create msbuild custom task modify c# code compile create custom msbuild task .cs file compiled csc.exe course modify place actual source file touched aware postsharp aop framework .net option project plus id learn precisely thanks richard,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
share build script lerna package lerna repo contains multiple package organised usual structure transpile package currently script package currently run build duplicate script every package define script place package currently linting script testing script root package sense effectively traversing whole monorepo looking test file lint seeem sense move build script well scoped individual package fact different colour output package unsatisfying solution create shell script root monorepo package file whilst feel right still involves duplication package requires setting permission shell script complicates ci better share command amongst package,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
url add home screen iphone safari customized add home screen page site url homepage saved example page httpwww.domain.comcategorypage.html add home screen save url httpwww.domain.com help greatly appreciated,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df5.reset_index(inplace=True)
df5.head()

,clean_concat,.net,ajax,algorithm,amazon-web-services,android,android-studio,angular,angularjs,array,...,ubuntu,unit-testing,visual-studio,web-services,window,winforms,wpf,xaml,xcode,xml
0,iterate equal value standard library suppose v...,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,integer division float multiplication calculat...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,create msbuild custom task modify c# code comp...,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,share build script lerna package lerna repo co...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,url add home screen iphone safari customized a...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
path = '/gdrive/My Drive/df6.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df5.to_csv(f, index=False)

In [ ]:
df5read=pd.read_csv("/gdrive/MyDrive/df6.csv", sep=',')
df5read.head()

,clean_concat,.net,ajax,algorithm,amazon-web-services,android,android-studio,angular,angularjs,array,...,ubuntu,unit-testing,visual-studio,web-services,window,winforms,wpf,xaml,xcode,xml
0,iterate equal value standard library suppose v...,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,integer division float multiplication calculat...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,create msbuild custom task modify c# code comp...,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,share build script lerna package lerna repo co...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,url add home screen iphone safari customized a...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.6 MB/s eta 0:00:00


In [ ]:
from datasets import *
dataset = load_dataset('csv', data_files="/gdrive/MyDrive/df6.csv")

train_testvalid = dataset['train'].train_test_split(test_size=0.2)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a8bef9a9b8e825ad/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_concat', '.net', 'ajax', 'algorithm', 'amazon-web-services', 'android', 'android-studio', 'angular', 'angularjs', 'array', 'asp.net', 'asp.net-core', 'asp.net-mvc', 'assembly', 'asynchronous', 'authentication', 'bash', 'c', 'c#', 'c++', 'c++11', 'class', 'cocoa', 'cocoa-touch', 'cs', 'database', 'debugging', 'django', 'docker', 'eclipse', 'entity-framework', 'file', 'firebase', 'gcc', 'git', 'google-chrome', 'gradle', 'hibernate', 'html', 'http', 'image', 'io', 'ios7', 'iphone', 'java', 'javascript', 'jpa', 'jquery', 'json', 'laravel', 'linq', 'linux', 'list', 'machine-learning', 'macos', 'maven', 'memory', 'multithreading', 'mysql', 'node.js', 'numpy', 'objective-c', 'optimization', 'panda', 'performance', 'php', 'postgresql', 'python', 'python-2.7', 'python-3.x', 'qt', 'r', 'reactjs', 'regex', 'rest', 'ruby', 'ruby-on-rails', 'scala', 'security', 'shell', 'spring', 'spring-boot', 'spring-mvc', 'sql', 'sql-server', 'ssl', 's

In [ ]:
example = dataset['train'][0]
example

{'clean_concat': 'odbcdbi r write table non-default schema r write table non-default schema package dbi writes rather writing exists ssms database reproducible examplewhat ive create table sql server non-default schema guest placing database sam weird result opened ssms lo behold table created help much appreciated',
 '.net': False,
 'ajax': False,
 'algorithm': False,
 'amazon-web-services': False,
 'android': False,
 'android-studio': False,
 'angular': False,
 'angularjs': False,
 'array': False,
 'asp.net': False,
 'asp.net-core': False,
 'asp.net-mvc': False,
 'assembly': False,
 'asynchronous': False,
 'authentication': False,
 'bash': False,
 'c': False,
 'c#': False,
 'c++': False,
 'c++11': False,
 'class': False,
 'cocoa': False,
 'cocoa-touch': False,
 'cs': False,
 'database': False,
 'debugging': False,
 'django': False,
 'docker': False,
 'eclipse': False,
 'entity-framework': False,
 'file': False,
 'firebase': False,
 'gcc': False,
 'git': False,
 'google-chrome': False

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['clean_concat']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['.net',
 'ajax',
 'algorithm',
 'amazon-web-services',
 'android',
 'android-studio',
 'angular',
 'angularjs',
 'array',
 'asp.net',
 'asp.net-core',
 'asp.net-mvc',
 'assembly',
 'asynchronous',
 'authentication',
 'bash',
 'c',
 'c#',
 'c++',
 'c++11',
 'class',
 'cocoa',
 'cocoa-touch',
 'cs',
 'database',
 'debugging',
 'django',
 'docker',
 'eclipse',
 'entity-framework',
 'file',
 'firebase',
 'gcc',
 'git',
 'google-chrome',
 'gradle',
 'hibernate',
 'html',
 'http',
 'image',
 'io',
 'ios7',
 'iphone',
 'java',
 'javascript',
 'jpa',
 'jquery',
 'json',
 'laravel',
 'linq',
 'linux',
 'list',
 'machine-learning',
 'macos',
 'maven',
 'memory',
 'multithreading',
 'mysql',
 'node.js',
 'numpy',
 'objective-c',
 'optimization',
 'panda',
 'performance',
 'php',
 'postgresql',
 'python',
 'python-2.7',
 'python-3.x',
 'qt',
 'r',
 'reactjs',
 'regex',
 'rest',
 'ruby',
 'ruby-on-rails',
 'scala',
 'security',
 'shell',
 'spring',
 'spring-boot',
 'spring-mvc',
 'sql',
 'sql-serv

PREPROCESSING

As models like BERT don't expect text as direct input, but rather input_ids, etc., we tokenize the text using the tokenizer. Here I'm using the AutoTokenizer API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' BCEWithLogitsLoss (which the model will use) will complain.

D'abord, on configure l'environnement d'execution

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17310348421836217294
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2307764796130884092
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 18 17:51:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |    373MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["clean_concat"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] odbcdbi r write table non - default schema r write table non - default schema package dbi writes rather writing exists ssms database reproducible examplewhat ive create table sql server non - default schema guest placing database sam weird result opened ssms lo behold table created help much appreciated [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['r', 'sql', 'sql-server']

In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch datasets.

In [ ]:
import torch

In [ ]:
encoded_dataset.set_format("torch")

MODELE DEFINING

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the problem_type to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely BCEWithLogitsLoss). We also make sure the output layer has len(labels) output neurons, and we set the id2label and label2id mappings

In [ ]:
from transformers import AutoModelForSequenceClassification
device = torch.device('cuda')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
TRAINING

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

1. TrainingArguments, which specify training hyperparameters. Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
2. a Trainer object.

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert_fine_tune",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a compute_metrics function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.3):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1051, 18939, 19797,  5638,  1054,  4339,  2795,  2512,  1011,
        12398,  8040, 28433,  1054,  4339,  2795,  2512,  1011, 12398,  8040,
        28433,  7427, 16962,  2072,  7009,  2738,  3015,  6526,  7020,  5244,
         7809, 16360, 14127, 21104,  2742,  2860, 12707,  4921,  2063,  3443,
         2795, 29296,  8241,  2512,  1011, 12398,  8040, 28433,  4113,  6885,
         7809,  3520,  6881,  2765,  2441,  7020,  5244,  8840, 27541,  2795,
         2580,  2393,  2172, 12315,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
#forward pass

outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0).to(device), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0).to(device))
outputs

SequenceClassifierOutput(loss=tensor(0.6839, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.3004, -0.4740, -0.0088, -0.5346,  0.2934, -0.0045,  0.4496,  0.5195,
         -0.0770,  0.0069,  0.4783, -0.4216, -0.1574,  0.1318,  0.2182, -0.3949,
         -0.1135,  0.5759, -0.6159, -0.0907, -0.3629,  0.0305,  0.0882, -0.6166,
         -0.1307, -0.5133, -0.0302,  0.0109, -0.0520,  0.1750, -0.2389, -0.4757,
          0.3918,  0.0581, -0.7026,  0.2248, -0.1055,  0.0267, -0.1427, -0.0260,
         -0.1305, -0.0129,  0.1229, -0.3431,  0.3236,  0.4430, -0.1738, -0.5267,
          0.2969, -0.2610,  0.0802, -0.2057, -0.0109,  0.0392, -0.5171,  0.2505,
         -0.2540,  0.2143,  0.2809,  0.2181,  0.3557,  0.0027,  0.4511, -0.1567,
         -0.1506,  0.6349, -0.6484, -0.3941, -0.1020,  0.0232, -0.3882, -0.1082,
         -0.0318, -0.3196,  0.0580,  0.0292,  0.1043,  0.0744, -0.3463,  0.1699,
         -0.1959, -0.5059, -0.0565,  0.2722,  0.3506, -0.1980, -

we start training:

In [ ]:
trainer2 = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer2.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.048800,0.053544,0.551971,0.745981,0.180000
2,0.043400,0.050906,0.577705,0.758857,0.218000
3,0.039900,0.048960,0.597062,0.779832,0.211000
4,0.037700,0.048385,0.594507,0.771697,0.227000
5,0.036500,0.048091,0.596369,0.774835,0.221000


TrainOutput(global_step=5000, training_loss=0.04167369728088379, metrics={'train_runtime': 1070.2963, 'train_samples_per_second': 37.373, 'train_steps_per_second': 4.672, 'total_flos': 2633425674240000.0, 'train_loss': 0.04167369728088379, 'epoch': 5.0})

EVALUATION

After the training, we evaluate our model on the validation set

In [ ]:
trainer2.evaluate(eval_dataset=encoded_dataset["test"])

{'eval_loss': 0.04896016791462898,
 'eval_f1': 0.5970619097586569,
 'eval_roc_auc': 0.7798323165556812,
 'eval_accuracy': 0.211,
 'eval_runtime': 6.9784,
 'eval_samples_per_second': 143.299,
 'eval_steps_per_second': 17.912,
 'epoch': 5.0}

In [ ]:
trainer2.evaluate(eval_dataset=encoded_dataset["valid"])

{'eval_loss': 0.049379706382751465,
 'eval_f1': 0.6014022331861855,
 'eval_roc_auc': 0.7804905363533735,
 'eval_accuracy': 0.221,
 'eval_runtime': 6.9476,
 'eval_samples_per_second': 143.935,
 'eval_steps_per_second': 17.992,
 'epoch': 5.0}

INFERENCE

we test the model on an exemple from the dataset

In [ ]:
text = "securing spring data repositoryrestresource crudrepository http internally spring data project us repositoryrestresource crudrepositorys expose entity rest able secure repository accessed http secured used internally service layer example spring security running adding annotation preauthorize crudrepository method cause security expression executed method service layer id glad someone point right direction edit ive removing rest export security annotation userrepository internally subclassing userrepository userrepositoryrestexported exporting securing seeing inconsistent implementation security annotation run wonder spring exporting userrepositoryrestexported sometimes userrepository times... edit code described edit userrepository.java userrepositoryrest.java"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer2.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 100])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example). The chosen threshold here is low because of the high number of labels.

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
#ou bien sélectionner les 5 premiers tags (ici si p > x)
predictions[np.where(probs >= 0.3)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)
##reel  = java spring security spring-boot

['java', 'spring', 'spring-boot']
